In [3]:
# Create Dataset.py so that we can load the CaravanaDataset and other things from it.

# <-- comment after files are created -->

# !jupyter nbconvert --to script 02_Dataset.ipynb --output CaravanaDataset
# !jupyter nbconvert --to script 03_Model.ipynb --output SegModel

[NbConvertApp] Converting notebook 03_Model.ipynb to script
[NbConvertApp] Writing 8725 bytes to SegModel.py


In [9]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import torch.optim as optim

import random
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from SegModel import UNet

from CaravanaDataset import (
    val_loader, train_loader, PROCESSED_IMAGES_DIR
)

from utils import (
    save_checkpoint,
    load_checkpoint,
    check_accuracy,
    save_preds_as_imgs
)

In [6]:
# -- Test Model if working properly --

def test():
    x = torch.randn((1, 3, 1792, 832))
    model = UNet(in_channels=3, out_channels=2)
    pred = model(x)
    print(x.shape)
    print(pred.shape)
    
test()

torch.Size([1, 3, 1792, 832])
torch.Size([1, 3, 1792, 832])


In [7]:
# Hyperparameters
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
NUM_EPOCHS = 1
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = True

In [20]:
# Training function will run over one batch and will return the training loss
def train_func(loader, model, optimizer, loss_fn, scalar):
    loop = tqdm(train_loader)
    for batch_idx, (images, targets) in enumerate(loop):
        images = images.permute([0, 3, 1, 2]).to(device=DEVICE)
        masks  = masks.unsqueeze(1).to(device=DEVICE)
        
        # Forward
        with torch.cuda.amp.autocast():
            preds = model(images)
            loss = loss_fn(preds, targets)
            
        # Backward
        optimizer.zero_grad()
        scalar.scale(loss).backward()
        scalar.step(optimizer)
        scalar.update()
        
        # update tqdm loop
        loop.set_postfix(loss=loss.item())
        
def main():
    ## set a model 
    model = UNet(in_channels=3, out_channels=2).to(device=DEVICE)
    
    loss_fn = nn.BCEWithLogitsLoss() # 2 classes 'Car' or 'Not car', if more than one class use cross entropy loss
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    ## loaders here ..
    
    scalar = torch.cuda.amp.GradScalar()
    for epoch in range(NUM_EPOCHS):
        train_func(train_loader, model, optimizer, loss_fn, scalar)
        
        # -- Save model --
        checkpoint = {
            "state_dict" : model.state_dict(), 
            "optimizer"  : optimizer.state_dict(),
        }
        
        save_checkpoint(checkpoint)
        
        # -- Check accuracy --
        check_accuracy(loader=val_loader, model=model, device=DEVICE)
        
        
        # print some examples to folder
        save_preds_as_imgs(val_loader, model=model,save_dir='pred_images', device=DEVICE)
        
        
        
        
    
    
    
    
        
        
        
        
        
    

    
